In [ ]:
using Compose
using Colors

In [ ]:
function render_grid_graph(G,paths::Vector{Vector{Int}}=Vector{Vector{Int}}();
        width=10,r=0.4,color="gray",pathcolors=["lime","red","blue","violet","orange","cyan"])
    # vertices
    x = Vector{Float64}()
    y = Vector{Float64}()
    for v in vertices(G)
        push!(x,get_prop(G,v,:x))
        push!(y,get_prop(G,v,:y))
    end
    x₀ = minimum(x) - r
    y₀ = minimum(y) - r
    Δx = maximum(x) + r - x₀
    Δy = maximum(y) + r - y₀
    # graphic size
    set_default_graphic_size((width)cm,(width*Δy/Δx)cm)
    # edges
    edge_list = []
    for e in edges(G)
        push!(edge_list, [(x[e.src],y[e.src]),(x[e.dst],y[e.dst])])
    end
    # paths
    rendered_paths = []
    starts = []
    goals = []
    while length(pathcolors) < length(paths)
        pathcolors = [pathcolors..., pathcolors...]
    end
    for (p,color) in zip(paths,pathcolors)
        rpath = []
        start = circle(x[p[1]],y[p[1]],r*0.8)
        goal = circle(x[p[end]],y[p[end]],r*0.8)
        for i in 1:length(p)-1
            push!(rpath, [(x[p[i]],y[p[i]]),(x[p[i+1]],y[p[i+1]])])
        end
        push!(rendered_paths,
            compose(context(),
                (context(),start,fill(color)),
                (context(),goal,fill(color)),
                (context(),line(rpath),stroke(color),linewidth(2pt))
            )
        )
    end
    
    compose(context(units=UnitBox(x₀, y₀, Δx, Δy)),
        rendered_paths...,
        compose(context(),circle(x,y,r*ones(length(x))),fill(color)),
        compose(context(),line(edge_list),stroke(color),linewidth(4pt))
    )
end

In [ ]:
using LightGraphs, MetaGraphs, GraphUtils
using NearestNeighbors
using CRCBS

In [ ]:
solver = CBS_Solver()
G = initialize_regular_grid_graph(;n_obstacles_x=1,n_obstacles_y=1)
mapf = initialize_mapf(
    CBS.LowLevelEnv(graph=G),
    [CBS.State(1,0),CBS.State(2,0)],
    [CBS.State(vtx=6),CBS.State(vtx=5)]
)
node = CBS.initialize_root_node(mapf)
solution, cost = CRCBS.solve!(solver,mapf);

In [ ]:
solution, cost = CRCBS.solve!(MetaAgentCBS_Solver(0),mapf);

In [ ]:
render_grid_graph(G,CBS.convert_to_vertex_lists(solution);width=12)

# Multi-Stage CBS

In [ ]:
# solver = MultiStageCBS.CBS_Solver()
solver = CBS_Solver()
G = initialize_regular_grid_graph(;n_obstacles_x=2,n_obstacles_y=2,obs_offset = [2;2])
mapf = initialize_mapf(
    MultiStageCBS.LowLevelEnv(graph=G),
    [   MultiStageCBS.State(vtx=1,stage=1,t=0),
        MultiStageCBS.State(vtx=2,stage=1,t=0),
        MultiStageCBS.State(vtx=26,stage=1,t=10),
        MultiStageCBS.State(vtx=167,stage=1,t=10),
        MultiStageCBS.State(vtx=41,stage=1,t=4)    ],
    [   [MultiStageCBS.State(vtx=6),MultiStageCBS.State(vtx=171)],
        [MultiStageCBS.State(vtx=5),MultiStageCBS.State(vtx=90)],
        [MultiStageCBS.State(vtx=100),MultiStageCBS.State(vtx=175)],
        [MultiStageCBS.State(vtx=85)],
        [MultiStageCBS.State(vtx=91),MultiStageCBS.State(vtx=1)]    ]
)

solution, cost = CRCBS.solve!(solver,mapf);

In [ ]:
# j = 1
# goal_vtxs = [s.vtx for s in mapf.goals[j]]
# path_vtxs = MultiStageCBS.convert_to_vertex_lists(solution[j])
# @show goal_vtxs;
# @show path_vtxs;

In [ ]:
render_grid_graph(G,MultiStageCBS.convert_to_vertex_lists(solution);width=12)

# Meta-Agent CBS

In [ ]:
# solver = CBS.CBSsolver()
solver = MetaAgentCBS_Solver(1)
G = initialize_regular_grid_graph(;n_obstacles_x=1,n_obstacles_y=1)
mapf = initialize_mapf(
    CBS.LowLevelEnv(graph=G),
    [CBS.State(1,0),CBS.State(2,0)],
    [CBS.State(vtx=6),CBS.State(vtx=5)]
)

solution, cost = CRCBS.solve!(solver,mapf);

In [ ]:
render_grid_graph(G,CBS.convert_to_vertex_lists(solution);width=12)

# Solve a Task Graphs Assignment and Routing Problem
Use the Gurobi MILP solver to get an assignment, then compute the paths using CBS

In [ ]:
# Task Graphs for scheduling real task sequences!
using TaskGraphs
using GraphPlottingBFS
using DataStructures
using JuMP, MathOptInterface
# using GLPK
using Gurobi

In [ ]:
N = 4                  # num robots
M = 8                  # num delivery tasks
# project_spec = construct_random_project_spec(M;max_parents=3,depth_bias=1.0,Δt_min=0,Δt_max=0)
project_spec = construct_random_project_spec(M;max_parents=3,depth_bias=0.25,Δt_min=0,Δt_max=0)
delivery_graph = construct_delivery_graph(project_spec,M)
r₀,s₀,sₜ,vtxs = initialize_random_2D_task_graph_env(N,M;vtxs=[[get_prop(G,v,:x),get_prop(G,v,:y)] for v in vertices(G)])
Drs, Dss = cached_pickup_and_delivery_distances(r₀,s₀,sₜ,(x1,x2)->gdistances(G,x1)[x2])
# initialize vector of operation times
DG = delivery_graph.graph
Δt = zeros(nv(DG)) # Δt[j] = wait time for object j to appear once inputs are satisfied
for op in project_spec.operations
    for id in get_output_ids(op)
        Δt[id] = duration(op)
    end
end
# set initial conditions
to0_ = Dict{Int,Float64}()
for v in vertices(DG)
    if is_leaf_node(DG,v)
        to0_[v] = 0.0
    end
end
tr0_ = Dict{Int,Float64}()
for i in 1:N
    tr0_[i] = 0.0
end

In [ ]:
plot_graph_bfs(project_spec.graph;mode=nothing,fillcolor="orange")

In [ ]:
plot_graph_bfs(delivery_graph.graph;mode=nothing)

In [ ]:
# solve task assignment problem
model = formulate_JuMP_optimization_problem(DG,Drs,Dss,Δt,to0_,tr0_,Gurobi.Optimizer;OutputFlag=0);
start_time = time()
optimize!(model)
solve_time = time() - start_time
optimal = (termination_status(model) == MathOptInterface.TerminationStatusCode(1))
@show solve_time
@show optimal;
assignment_matrix = Matrix{Int}(value.(model[:x]));
# process solution
spec = TaskGraphProblemSpec(N,M,DG,Drs,Dss,Δt,tr0_,to0_)
cache = SearchCache(N,M,to0_,tr0_)
for j in 1:M
    i = findfirst(assignment_matrix[:,j] .== 1)
    cache.x[i,j] = 1
end
cache = process_solution(model,cache,spec);
# construct task sequences
task_sequences = Dict{Int,Vector{Int}}()
for i in 1:N
    robot_id = i
    seq = Vector{Int}()
    for j in 1:M
        if cache.x[i,j] == 1
            push!(seq, j)
            i = j + N
        end
    end
    task_sequences[robot_id] = seq
end

In [ ]:
start_states = Vector{MultiStageCBS.State}()
goal_sequences = Vector{Vector{MultiStageCBS.State}}()
for i in 1:N
    push!(start_states, MultiStageCBS.State(vtx=r₀[i], stage=1, t=Int(cache.tr0[i])))
    goals = Vector{MultiStageCBS.State}()
    for j in task_sequences[i]
        push!(goals, MultiStageCBS.State(vtx=s₀[j])) # object start location 
        push!(goals, MultiStageCBS.State(vtx=sₜ[j])) # object destination
    end
    push!(goal_sequences, goals)
end
mapf = MultiMAPF(G, start_states, goal_sequences)
# node = initialize_root_node(mapf)
solution, cost = CRCBS.solve!(solver,mapf);

In [ ]:
render_grid_graph(G,MultiStageCBS.convert_to_vertex_lists(solution);width=12)

# Priority-based CBS
In this situation we care about optimizing the makespan rather than the sum of individual travel times. The structure of the task dependency tree and the assignments means that some deliveries are "on the critical path", others are close to the critical path, and others have lots of "wiggle room".